<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Regression Models with Keras Assignment</font></h1>

<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Regression Models with Keras</font></h1>

## Download and Clean Dataset

Let's download the data and read it into a <em>pandas</em> dataframe.


First we start by importing the pandas (for dataframe creation) and the Numpy libraries (for scientific computing).

In [1]:
import pandas as pd
import numpy as np

Let's download the data and read it into a <em>pandas</em> dataframe.

In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


#### Let's check how many data points we have.

In [3]:
concrete_data.shape

(1030, 9)

Next, we check our dataset for any missing values

In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

Perfect! We seen to have a clean dataset with no missing values.

The next step is to split the predictors from the target feature

1. Randomly split the data into a training and test sets by holding 30% of the data for testing

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
train_set, test_set = train_test_split(concrete_data,test_size=0.30)

In [8]:
print('train {0}'.format(train_set.shape))

train (721, 9)


In [9]:
print('test {0}'.format(test_set.shape))

test (309, 9)


## The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.

## Let's split the sets up for mse

In [10]:
# need to use train_set and test_set
concrete_data_columns = train_set.columns

In [11]:
predictors_train_set = train_set[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength

In [12]:
target_train_set = train_set['Strength'] # Strength column

In [13]:
#test set
predictors_test_set = test_set[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target_test_set = test_set['Strength'] # Strength column

The last data preparation step is to normalize our predtictors.

In [14]:
predictors_norm_train_set = (predictors_train_set - predictors_train_set.mean()) / predictors_train_set.std()
predictors_norm_test_set = (predictors_test_set - predictors_test_set.mean()) / predictors_test_set.std()

In [15]:
predictors_norm_train_set.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
408,-0.972785,0.643636,1.032962,-0.067590,-0.411355,0.420701,-0.369614,-0.681424
457,-0.261000,-0.851976,1.028237,0.533134,-0.037027,0.896335,-0.276605,0.196560
409,-1.075577,0.644788,1.190475,-0.270913,0.303271,0.407598,-0.372094,-0.681424
819,2.392195,-0.851976,-0.835135,0.352917,-1.023892,1.962906,-2.028901,0.759796
783,0.171502,-0.851976,-0.835135,0.491546,-1.023892,1.438793,-0.143911,0.759796


Let's save the number of predictors to n_cols since we will need this number when building our network.

In [16]:
num_cols = predictors_norm_train_set.shape[1] # number of predictors
print( num_cols )

8


#### Let's go ahead and import the Keras library

In [17]:
import keras

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

As you can see, the TensorFlow backend was used to install the Keras library.

Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.

In [18]:
from keras.models import Sequential
from keras.layers import Dense

 Now let us build our baseline model. The requirements according to the assignment instruction is to use the Keras library to build a neural network with one hidden layer which contains 10 nodes. We are to use the ReLU function for activating this model. Furthermore, we are to use the adam optimizer and the mean squared error as the loss function. Let's go ahead and do just that.

In [19]:
# Let's define our regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(num_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function creates a model that has one hidden layer, each of 10 hidden units.

2. A Train the model on the training data using 50 epochs

3. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.

4. Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors."

In [20]:
# get the list for mse for printing
import statistics 

In [21]:
from sklearn.metrics import mean_squared_error

In [22]:
# build the model
model = regression_model()

In [23]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                90        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
mse_results=[]

In [25]:
for x in range(50):
     # fit the model
    model.fit(predictors_train_set, target_train_set, epochs=50, verbose=0)
    train_results = model.predict( predictors_test_set )
    mse_results.append( mean_squared_error(target_test_set, train_results) )

5. Report the mean and the standard deviation of the mean squared errors.

In [34]:
print(statistics.mean(mse_results)) 
print(statistics.stdev( mse_results))

52.899856240977485
24.70798635561496


## B.Normalize the data
Repeat Part A but use a normalized version of the data. Recall that one way to normalize the data is by subtracting the mean from the individual predictors and dividing by the standard deviation.

We already have our data normalized refer to line for more information

In [35]:
for x in range(50):
     # fit the model
    model.fit(predictors_norm_train_set, target_train_set, epochs=50, verbose=0)
    train_results = model.predict( predictors_norm_test_set )
    mse_results.append( mean_squared_error(target_test_set, train_results) )

In [37]:
print(statistics.mean( mse_results ))
print( statistics.stdev( mse_results ))

52.855219519859205
34.97369061453728


Conclusion: There is not much of a difference in the MSE but the standard deviation seems to increase with normalization

## C. Increase the number of epochs
Repeat Part B but use 100 epochs this time for training.

In [38]:
for x in range(50):
     # fit the model
    model.fit(predictors_norm_train_set, target_train_set, epochs=100, verbose=0)
    train_results = model.predict( predictors_norm_test_set )
    mse_results.append( mean_squared_error(target_test_set, train_results) )
    

In [36]:
print(statistics.mean( mse_results ))
print( statistics.stdev( mse_results ))

52.855219519859205
34.97369061453728


## D. Increase the number of hidden layers

Repeat part B but use a neural network with the following instead:

- Three hidden layers, each of 10 nodes and ReLU activation function.

In [39]:
# Let's define our regression model
def regression_model_three_layers():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(num_cols,)))
    model.add(Dense(10, activation='relu', input_shape=(num_cols,)))
    model.add(Dense(10, activation='relu', input_shape=(num_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [40]:
# get the list for mse for printing
import statistics 

In [41]:
from sklearn.metrics import mean_squared_error

In [42]:
# build the model
model = regression_model_three_layers()

In [43]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 10)                90        
_________________________________________________________________
dense_4 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_5 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________
None


In [44]:
mse_results=[]

In [45]:
for x in range(50):
     # fit the model
    model.fit(predictors_norm_train_set, target_train_set, epochs=50, verbose=0)
    train_results = model.predict( predictors_norm_test_set )
    mse_results.append( mean_squared_error(target_test_set, train_results) )

In [46]:
print(statistics.mean(mse_results))
print(statistics.stdev(mse_results))

42.815229782071164
17.48566384697234


Conclusion: With increasing the hidden layers, reuslts in a significant decrease in both MSE and the standard deviation.